In [1]:
# i want to use ollam and model llama3.1:latest and in-context learning to do credit card fraud detection. And test the performance of the model.

# the huggingface dataset for legitimate transactions is called "yunfan-y/fraud-detection-legitimate"

# the huggingface dataset for fraudulent transactions is called "yunfan-y/fraud-detection-fraud"

# those datasets have train, validation, and test splits. You should use the test split to test the performance of the model.

# all datasets have columns "conversation" and "response" 
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: LEGITIMATE

In [2]:
from datasets import load_dataset, concatenate_datasets

# Load legitimate transactions (test split)
legitimate_test = load_dataset("yunfan-y/fraud-detection-legitimate", split="test")

# Load fraudulent transactions (test split)
fraud_test = load_dataset("yunfan-y/fraud-detection-fraud", split="test")

# Combine the datasets
test_dataset = concatenate_datasets([legitimate_test, fraud_test])

# Shuffle the dataset
test_dataset = test_dataset.shuffle(seed=42)

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ... existing dataset loading code ...

from ollama import Client
import random
from sklearn.metrics import classification_report, confusion_matrix
import time

# Initialize Ollama client
client = Client(host='http://localhost:11434')

# Function to create prompt with in-context examples
def create_prompt(test_example, num_examples=5):
    # Randomly sample examples from training set (excluding the test example)
    train_examples = test_dataset.select(
        random.sample(range(len(test_dataset)), num_examples)
    )
    
    # Build prompt with examples
    prompt = "You are a fraud detection system. Analyze these transactions and determine if they are LEGITIMATE or FRAUD. Reply with either LEGITIMATE or FRAUD only.\n\n"
    
    # Add training examples
    for example in train_examples:
        prompt += f"Transaction: {example['conversation']}\nResult: {example['response']}\n\n"
    
    # Add test example
    prompt += f"Transaction: {test_example['conversation']}\nResult:"
    
    return prompt

# Function to evaluate model
def evaluate_model(num_samples=100):
    results = []
    true_labels = []
    
    # Sample subset for testing
    test_subset = test_dataset.select(range(num_samples))
    
    for idx, example in enumerate(test_subset):
        try:
            # Create prompt with in-context examples
            prompt = create_prompt(example)
            
            # Get model prediction
            response = client.generate(
                model='llama3.1:latest',
                prompt=prompt,
                options={
                    'num_predict': 10,  # replaced max_tokens with num_predict
                    'temperature': 0.1  # adding low temperature for more consistent results
                }
            )
            
            # Extract prediction (assuming model outputs LEGITIMATE or FRAUD)
            prediction = 'LEGITIMATE' if 'LEGITIMATE' in response['response'] else 'FRAUD'
            
            results.append(prediction)
            true_labels.append(example['response'])
            
            # Print progress
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{num_samples} examples")
                
            # Small delay to avoid rate limiting
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Error processing example {idx}: {e}")
            continue
    
    # Print performance metrics
    print("\nClassification Report:")
    print(classification_report(true_labels, results))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, results))
    
    return results, true_labels

# Run evaluation
results, true_labels = evaluate_model(num_samples=500)

Processed 10/500 examples
Processed 20/500 examples
Processed 30/500 examples
Processed 40/500 examples
Processed 50/500 examples
Processed 60/500 examples
Processed 70/500 examples
Processed 80/500 examples
Processed 90/500 examples
Processed 100/500 examples
Processed 110/500 examples
Processed 120/500 examples
Processed 130/500 examples
Processed 140/500 examples
Processed 150/500 examples
Processed 160/500 examples
Processed 170/500 examples
Processed 180/500 examples
Processed 190/500 examples
Processed 200/500 examples
Processed 210/500 examples
Processed 220/500 examples
Processed 230/500 examples
Processed 240/500 examples
Processed 250/500 examples
Processed 260/500 examples
Processed 270/500 examples
Processed 280/500 examples
Processed 290/500 examples
Processed 300/500 examples
Processed 310/500 examples
Processed 320/500 examples
Processed 330/500 examples
Processed 340/500 examples
Processed 350/500 examples
Processed 360/500 examples
Processed 370/500 examples
Processed 

In [4]:
# ... existing dataset loading code ...

from ollama import Client
import random
from sklearn.metrics import classification_report, confusion_matrix
import time

# Initialize Ollama client
client = Client(host='http://localhost:11434')

# Function to create prompt with in-context examples
def create_prompt(test_example, num_examples=5):
    # Randomly sample examples from training set (excluding the test example)
    train_examples = test_dataset.select(
        random.sample(range(len(test_dataset)), num_examples)
    )
    
    # Build prompt with examples
    prompt = "You are a fraud detection system. Analyze these transactions and determine if they are LEGITIMATE or FRAUD. Reply with either LEGITIMATE or FRAUD only.\n\n"
    
    # Add training examples
    for example in train_examples:
        prompt += f"Transaction: {example['conversation']}\nResult: {example['response']}\n\n"
    
    # Add test example
    prompt += f"Transaction: {test_example['conversation']}\nResult:"
    
    return prompt

# Function to evaluate model
def evaluate_model(num_samples=100):
    results = []
    true_labels = []
    
    # Sample subset for testing
    test_subset = test_dataset.select(range(num_samples))
    
    for idx, example in enumerate(test_subset):
        try:
            # Create prompt with in-context examples
            prompt = create_prompt(example)
            
            # Get model prediction
            response = client.generate(
                model='gemma2:latest',
                prompt=prompt,
                options={
                    'num_predict': 10,  # replaced max_tokens with num_predict
                    'temperature': 0.1  # adding low temperature for more consistent results
                }
            )
            
            # Extract prediction (assuming model outputs LEGITIMATE or FRAUD)
            prediction = 'LEGITIMATE' if 'LEGITIMATE' in response['response'] else 'FRAUD'
            
            results.append(prediction)
            true_labels.append(example['response'])
            
            # Print progress
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{num_samples} examples")
                
            # Small delay to avoid rate limiting
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Error processing example {idx}: {e}")
            continue
    
    # Print performance metrics
    print("\nClassification Report:")
    print(classification_report(true_labels, results))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, results))
    
    return results, true_labels

# Run evaluation
results, true_labels = evaluate_model(num_samples=500)

Processed 10/500 examples
Processed 20/500 examples
Processed 30/500 examples
Processed 40/500 examples
Processed 50/500 examples
Processed 60/500 examples
Processed 70/500 examples
Processed 80/500 examples
Processed 90/500 examples
Processed 100/500 examples
Processed 110/500 examples
Processed 120/500 examples
Processed 130/500 examples
Processed 140/500 examples
Processed 150/500 examples
Processed 160/500 examples
Processed 170/500 examples
Processed 180/500 examples
Processed 190/500 examples
Processed 200/500 examples
Processed 210/500 examples
Processed 220/500 examples
Processed 230/500 examples
Processed 240/500 examples
Processed 250/500 examples
Processed 260/500 examples
Processed 270/500 examples
Processed 280/500 examples
Processed 290/500 examples
Processed 300/500 examples
Processed 310/500 examples
Processed 320/500 examples
Processed 330/500 examples
Processed 340/500 examples
Processed 350/500 examples
Processed 360/500 examples
Processed 370/500 examples
Processed 